<a href="https://colab.research.google.com/github/leeds1219/VQA_retriever/blob/main/MAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install higher transformers torch torchvision

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import CLIPProcessor, CLIPModel
import higher  # MAML을 위한 라이브러리


# 모델 정의
class CaptionSearchModel(nn.Module):
    def __init__(self, pretrained_model="openai/clip-vit-base-patch32"):
        super(CaptionSearchModel, self).__init__()
        # CLIP 모델 로드
        self.clip = CLIPModel.from_pretrained(pretrained_model)
        self.processor = CLIPProcessor.from_pretrained(pretrained_model)

        # CLIP 모델의 모든 파라미터를 frozen 상태로 설정 (이미지 인코더와 텍스트 인코더 포함)
        for param in self.clip.parameters():
            param.requires_grad = False

        # 학습 가능한 캡션 텍스트 인코더
        # 캡션 텍스트 인코더는 텍스트 모델을 그대로 사용하되, 이 부분은 학습이 가능하도록 설정
        self.caption_text_encoder = self.clip.text_model
        for param in self.caption_text_encoder.parameters():
            param.requires_grad = True  # 텍스트 인코더는 학습 가능

    def forward(self, text_inputs, image):
        # CLIP을 사용해 텍스트와 이미지 임베딩 생성 (학습되지 않음)
        inputs = self.processor(text=text_inputs, images=image, return_tensors="pt", padding=True)
        outputs = self.clip(**inputs)

        text_embedding = outputs.text_embeds  # 텍스트 임베딩
        image_embedding = outputs.image_embeds  # 이미지 임베딩

        # 텍스트와 이미지 임베딩을 concat
        combined_embedding = torch.cat((text_embedding, image_embedding), dim=-1)
        return combined_embedding

    def compute_similarity(self, query_embedding, caption_embedding):
        """
        주어진 query 임베딩과 caption 임베딩 간의 유사도 계산
        """
        query_embedding = F.normalize(query_embedding, p=2, dim=-1)
        caption_embedding = F.normalize(caption_embedding, p=2, dim=-1)

        similarity = torch.matmul(query_embedding, caption_embedding.T)
        return similarity


# Contrastive Loss 정의
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, positive_similarity, negative_similarity):
        """
        Contrastive loss 계산
        """
        loss_pos = torch.clamp(positive_similarity, min=0)
        loss_neg = torch.clamp(self.margin - negative_similarity, min=0)
        loss = loss_pos + loss_neg
        return loss.mean()


# 학습기 정의
class CaptionSearchTrainer:
    def __init__(self, model, tokenizer, lr=1e-5, margin=1.0):
        self.model = model
        self.tokenizer = tokenizer
        self.optimizer = optim.Adam(model.parameters(), lr=lr)
        self.criterion = ContrastiveLoss(margin=margin)

    def train_step(self, query, image, positive_caption, negative_captions):
        # 텍스트와 이미지 임베딩을 생성
        query_embedding = self.model(query, image)

        # Positive Caption 임베딩 생성
        positive_caption_embedding = self.model(positive_caption, image)

        # Negative Caption 임베딩 생성 (배치 내 다른 캡션들은 negative)
        negative_caption_embeddings = []
        for neg_caption in negative_captions:
            negative_caption_embeddings.append(self.model(neg_caption, image))
        negative_caption_embeddings = torch.stack(negative_caption_embeddings, dim=0)

        # Similarity 계산
        positive_similarity = self.model.compute_similarity(query_embedding, positive_caption_embedding)

        # Negative similarity는 배치 내의 다른 샘플들과 비교
        negative_similarities = []
        for neg_embedding in negative_caption_embeddings:
            negative_similarities.append(self.model.compute_similarity(query_embedding, neg_embedding))

        negative_similarities = torch.stack(negative_similarities, dim=0)

        # Contrastive loss 계산
        loss = self.criterion(positive_similarity, negative_similarities)
        return loss

    def meta_train_step(self, support_query, support_image, support_positive_caption, support_negative_captions, query_query, query_image, query_positive_caption, query_negative_captions):
        """
        MAML의 inner loop와 outer loop를 구현하는 함수
        """
        # higher 라이브러리를 사용하여 모델 파라미터를 복사합니다.
        # copy_initial_weights=True는 모델 파라미터를 복사하여 inner loop에서 업데이트할 수 있도록 합니다.
        with higher.innerloop_ctx(self.model, self.optimizer, copy_initial_weights=True) as (fmodel, diffopt):
            # Support set을 사용하여 모델을 학습 (inner loop)
            support_loss = self.train_step(support_query, support_image, support_positive_caption, support_negative_captions)
            diffopt.step(support_loss)  # inner loop에서 gradient 업데이트

            # Query set을 사용하여 평가 (outer loop)
            query_loss = self.train_step(query_query, query_image, query_positive_caption, query_negative_captions)

        return query_loss

    def train(self, dataloader, num_epochs=10):
        self.model.train()
        for epoch in range(num_epochs):
            for batch in dataloader:
                # 배치를 Support set과 Query set으로 나눕니다
                support_query, support_image, support_positive_caption, support_negative_captions, query_query, query_image, query_positive_caption, query_negative_captions = batch

                self.optimizer.zero_grad()

                # MAML을 통한 메타 학습
                loss = self.meta_train_step(support_query, support_image, support_positive_caption, support_negative_captions, query_query, query_image, query_positive_caption, query_negative_captions)

                loss.backward()  # gradient 계산
                self.optimizer.step()  # 메타 학습을 위한 파라미터 업데이트

                print(f"Epoch {epoch+1}, Loss: {loss.item()}")


# 예시 데이터 (실제로는 데이터셋을 불러와야 함)
captions = ["A man is holding a dog", "A car is parked on the road", "A person is riding a bicycle", "A dog is playing in the park"]
question = "What is the person doing?"
image = torch.randn(1, 3, 224, 224)  # 가상의 이미지 Tensor

# 모델 초기화
model = CaptionSearchModel()

# 모델과 토크나이저, 학습기 초기화
tokenizer = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
trainer = CaptionSearchTrainer(model, tokenizer)

# 가상의 데이터 로더
from torch.utils.data import DataLoader, TensorDataset

max_length = 77  # CLIP 모델에서 사용하는 최대 길이 (텍스트 길이를 77로 고정)

# Tokenizer를 사용하여 텍스트를 토큰화한 뒤, 텐서로 변환 (padding과 truncation 적용)
query_tensor = tokenizer(question, return_tensors="pt", padding='max_length', truncation=True, max_length=max_length)
positive_tensor = tokenizer(captions[0], return_tensors="pt", padding='max_length', truncation=True, max_length=max_length)
negative_tensor = [tokenizer(caption, return_tensors="pt", padding='max_length', truncation=True, max_length=max_length) for caption in captions[1:]]

# 이미지는 Tensor로 변환하여 사용
image_tensor = torch.randn(1, 3, 224, 224)

# Dataset과 DataLoader 생성
dataset = TensorDataset(query_tensor['input_ids'], image_tensor, positive_tensor['input_ids'],
                        torch.stack([neg['input_ids'] for neg in negative_tensor], dim=0))

dataloader = DataLoader(dataset, batch_size=1)

# 학습 시작
trainer.train(dataloader, num_epochs=10)



In [16]:
query_tensor

{'input_ids': tensor([[49406,   768,   533,   518,  2533,  1960,   286, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])}

In [15]:
positive_tensor

{'input_ids': tensor([[49406,   320,   786,   533,  5050,   320,  1929, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])}

In [17]:
negative_tensor

[{'input_ids': tensor([[49406,   320,  1615,   533, 16487,   525,   518,  1759, 49407, 49407,
          49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
          49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
          49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
          49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
          49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
          49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
          49407, 49407, 49407, 49407, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]])},
 {'input_ids': tensor([[49406,   320,  2533,   533,  6765,   320, 11652,

In [18]:
query_tensor['input_ids']

tensor([[49406,   768,   533,   518,  2533,  1960,   286, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]])

In [19]:
positive_tensor['input_ids']

tensor([[49406,   320,   786,   533,  5050,   320,  1929, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]])